In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [103]:
df = pd.read_csv('f2.csv')
df

,Temparature,Humidity,Moisture,Soil_Type,Crop_Type,Nitrogen,Potassium,Phosphorous,Fertilizer
0,20,83,26,Clayey,rice,90,49,36,Urea
1,25,84,32,Loamy,rice,66,59,36,Urea
2,33,64,50,Loamy,Wheat,41,0,0,Urea
3,34,65,54,Loamy,Wheat,38,0,0,Urea
4,38,72,51,Loamy,Wheat,39,0,0,Urea
...,...,...,...,...,...,...,...,...,...
547,12,95,42,Sandy,orange,11,14,5,10-10-10
548,12,95,42,Sandy,orange,11,14,5,10-10-10
549,27,80,31,Clayey,kidneybeans,19,21,6,10-10-10
550,27,80,31,Clayey,kidneybeans,19,21,6,10-10-10


In [104]:

df.drop("Moisture",axis=1,inplace =True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Temparature  552 non-null    int64 
 1   Humidity     552 non-null    int64 
 2   Soil_Type    552 non-null    object
 3   Crop_Type    552 non-null    object
 4   Nitrogen     552 non-null    int64 
 5   Potassium    552 non-null    int64 
 6   Phosphorous  552 non-null    int64 
 7   Fertilizer   552 non-null    object
dtypes: int64(5), object(3)
memory usage: 34.6+ KB


In [112]:
df.drop("Soil_Type",inplace=True,axis=1)
df['Fertilizer'].value_counts()

Urea                  108
DAP                   104
28-28                  68
20-20                  56
14-35-14               56
TSP                    28
17-17-17               28
10-26-26               28
15-15-15               16
14-14-14               16
10-10-10               16
Superphosphate         12
Potassium sulfate.     12
Potassium chloride      4
Name: Fertilizer, dtype: int64

In [113]:
df['Crop_Type'].value_counts()

Cotton         64
Sugarcane      52
Millets        44
Pulses         40
Paddy          40
Wheat          36
pomegranate    32
Ground Nuts    28
Barley         28
orange         28
Oil seeds      28
Tobacco        28
Maize          24
rice           24
watermelon     20
kidneybeans    20
coffee         16
Name: Crop_Type, dtype: int64

In [114]:
y = df['Fertilizer'].copy()
X = df.drop(['Fertilizer'],axis=1).copy()

In [115]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,shuffle=True,random_state=1)

In [119]:
X_train.head()

,Temparature,Humidity,Crop_Type,Nitrogen,Potassium,Phosphorous
410,27,54,Pulses,13,0,13
342,34,65,Wheat,23,0,19
549,27,80,kidneybeans,19,21,6
223,26,82,rice,86,41,36
80,29,58,Ground Nuts,24,0,23


In [137]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import LinearSVC


# Define the transformation for nominal (categorical) features
nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse=False))
])

# Define the transformation for numeric features (scaling)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Preprocessor to handle both categorical and numeric features
preprocessor = ColumnTransformer(transformers=[
    ('nominal', nominal_transformer, ['Crop_Type']),  # Apply one-hot encoding to 'Crop_Type'
    ('numeric', numeric_transformer, ['Temparature', 'Humidity', 'Nitrogen', 'Potassium', 'Phosphorous'])  # Apply scaling to numeric columns
], remainder='passthrough')

# Complete pipeline with preprocessor and classifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LinearSVC(max_iter=10000))  # Use LinearSVC instead of RandomForestClassifier
])

# Now you can fit the model on your training data
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('nominal',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(sparse=False))]),
                                                  ['Crop_Type']),
                                                 ('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Temparature', 'Humidity',
                                                   'Nitrogen', 'Potassium',
                                                   'Phosphorous'])])),
                ('classifier', LinearSVC(max_iter=10000))])

In [138]:
# model=LinearSVC()
# model.fit(X_train,y_train)

In [139]:
model.score(X_test,y_test)

0.8253012048192772

In [140]:
from sklearn.metrics import classification_report
print(classification_report(y_test,model.predict(X_test)))

                    precision    recall  f1-score   support

          10-10-10       0.00      0.00      0.00         5
          10-26-26       1.00      0.64      0.78        14
          14-14-14       0.38      1.00      0.55         3
          14-35-14       0.85      1.00      0.92        17
          15-15-15       1.00      0.50      0.67         6
          17-17-17       0.85      1.00      0.92        11
             20-20       1.00      1.00      1.00        16
             28-28       1.00      1.00      1.00        20
               DAP       0.79      0.81      0.80        27
Potassium chloride       0.00      0.00      0.00         2
Potassium sulfate.       0.33      0.33      0.33         3
    Superphosphate       0.75      0.60      0.67         5
               TSP       0.36      0.50      0.42         8
              Urea       1.00      0.97      0.98        29

          accuracy                           0.83       166
         macro avg       0.66      0.6

C:\Users\haris\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\haris\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\haris\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [148]:
import numpy as np
import pandas as pd

# Example new input value (replace this with your actual values)
new_data = pd.DataFrame({
    'Temparature': [21],  # Example numeric value
    'Humidity': [82],  # Example numeric value
    'Crop_Type': ['rice'],  # Example categorical value
    'Nitrogen': [90],  # Example numeric value
    'Potassium': [42],  # Example numeric value
    'Phosphorous': [20]  # Example numeric value
})

# Get the decision function values for the new input
decision_values = model.decision_function(new_data)

# Get the class labels corresponding to the decision values
classes = model.named_steps['classifier'].classes_

# Get the indices of the top 3 decision values (highest confidence)
top_3_indices = np.argsort(decision_values[0])[-3:]  # Get indices of top 3 confidence values

# Get the corresponding class labels for the top 3 decision values
top_3_classes = classes[top_3_indices]
top_3_confidences = decision_values[0, top_3_indices]

# Print the top 3 predictions and their confidence scores
for i in range(3):
    print(f"Prediction {i+1}: {top_3_classes[i]}")


Prediction 1: TSP
Prediction 2: 10-10-10
Prediction 3: Urea
